In [1]:
import torch
from torch.utils.data import DataLoader, Dataset, TensorDataset
from torchvision import models, datasets, transforms
from PIL import Image
import os
import zipfile
import torch.optim as optim
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np

# Load experimental data
def load_all_experimental_data(test_digits_folder):
    train_images = []
    train_labels = []
    test_images = []
    test_labels = []
    participant_data = {}

    transform = transforms.Compose([
        transforms.Resize((16, 16)),
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])

    for filename in os.listdir(test_digits_folder):
        if filename.endswith('.zip') and filename.startswith('experiment_results_participant'):
            participant_number = int(filename.split('participant')[1].split('.')[0])
            zip_filepath = os.path.join(test_digits_folder, filename)

            participant_train_images = []
            participant_train_labels = []
            participant_test_images = []
            participant_test_labels = []

            with zipfile.ZipFile(zip_filepath, 'r') as zip_ref:
                for img_filename in zip_ref.namelist():
                    if img_filename.endswith('.png'):
                        with zip_ref.open(img_filename) as file:
                            img = Image.open(file).convert('L')
                            img_tensor = transform(img)
                            
                            digit = int(img_filename.split('_')[0])
                            
                            if 'composite' in img_filename:
                                test_images.append(img_tensor)
                                test_labels.append(digit)
                                participant_test_images.append(img_tensor)
                                participant_test_labels.append(digit)
                            else:
                                train_images.append(img_tensor)
                                train_labels.append(digit)
                                participant_train_images.append(img_tensor)
                                participant_train_labels.append(digit)

            participant_data[participant_number] = {
                'train': (torch.stack(participant_train_images), torch.tensor(participant_train_labels)),
                'test': (torch.stack(participant_test_images), torch.tensor(participant_test_labels))
            }

    return (torch.stack(train_images), torch.tensor(train_labels), 
            torch.stack(test_images), torch.tensor(test_labels),
            participant_data)

# Load experimental data
exp_train_images, exp_train_labels, exp_test_images, exp_test_labels, participant_data = load_all_experimental_data('test_digits')

In [7]:
from sklearn.model_selection import train_test_split

# Split the test data into validation and test sets
val_images, final_test_images, val_labels, final_test_labels = train_test_split(
    exp_test_images, exp_test_labels, test_size=0.5, random_state=42
)

# Assuming exp_train_images and exp_train_labels are already loaded
train_dataset = TensorDataset(exp_train_images, exp_train_labels)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Create DataLoader for validation set
val_dataset = torch.utils.data.TensorDataset(val_images, val_labels)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=True)

# Create DataLoader for final test set
test_dataset = torch.utils.data.TensorDataset(final_test_images, final_test_labels)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [2]:
import torch.nn as nn

# Define LeNet5 architecture modified for 16x16 images
class LeNet5_16x16(nn.Module):
    def __init__(self, num_classes=10):
        super(LeNet5_16x16, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 6, kernel_size=5),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(6, 16, kernel_size=5),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2)
        )
        self.fc1 = nn.Linear(16 * 1 * 1, 120)  # Adjusted for 16x16 input size
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, num_classes)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)  # Flatten the tensor
        out = self.fc1(out)
        out = self.fc2(out)
        out = self.fc3(out)
        return out

# Load LeNet5 model for 16x16 images
def load_lenet5_model_16x16():
    model = LeNet5_16x16(num_classes=10)
    return model

In [4]:
# Fine-tuning function for LeNet5 on your dataset
def fine_tune_lenet(model, train_loader, val_loader=None, num_epochs=15, save_path="lenet5_model.pth"):
    criterion = nn.CrossEntropyLoss()  # Loss function for classification tasks

    optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam optimizer

    model.train()  # Set model to training mode
    
    for epoch in range(num_epochs):
        running_loss = 0.0
        
        # Training loop
        for images, labels in train_loader:
            optimizer.zero_grad()  # Zero the parameter gradients
            
            outputs = model(images)  # Forward pass
            loss = criterion(outputs, labels)  # Compute loss
            
            loss.backward()  # Backward pass (compute gradients)
            optimizer.step()  # Update weights
            
            running_loss += loss.item()
        
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss:.4f}')
        
        # Validation loop (if validation set is provided)
        if val_loader:
            correct_val = 0
            total_val = 0
            model.eval()  # Set model to evaluation mode during validation
            
            with torch.no_grad():  # Disable gradient computation during validation
                for val_images_batch, val_labels_batch in val_loader:
                    outputs_val = model(val_images_batch)
                    _, predicted_val = torch.max(outputs_val.data, 1)   # Get predicted class
                    
                    total_val += val_labels_batch.size(0)
                    correct_val += (predicted_val == val_labels_batch).sum().item()
            
            print(f'Validation Accuracy: {100 * correct_val / total_val:.2f}%')
            
            model.train()  # Switch back to training mode after validation

    # Save the trained model after training is complete
    torch.save(model.state_dict(), save_path)
    print(f"Model saved to {save_path}")

# Example usage:
lenet_model_16x16 = load_lenet5_model_16x16()
fine_tune_lenet(lenet_model_16x16, train_loader=train_loader, val_loader=val_loader, save_path="lenet5_trained_model.pth")

Epoch [1/15], Loss: 2408.0627
Validation Accuracy: 68.42%
Epoch [2/15], Loss: 2377.3934
Validation Accuracy: 73.68%
Epoch [3/15], Loss: 2365.0198
Validation Accuracy: 75.79%
Epoch [4/15], Loss: 2358.5327
Validation Accuracy: 85.26%
Epoch [5/15], Loss: 2354.6844
Validation Accuracy: 90.53%
Epoch [6/15], Loss: 2351.0160
Validation Accuracy: 90.53%
Epoch [7/15], Loss: 2346.9722
Validation Accuracy: 74.74%
Epoch [8/15], Loss: 2344.3531
Validation Accuracy: 86.32%
Epoch [9/15], Loss: 2342.2535
Validation Accuracy: 88.42%
Epoch [10/15], Loss: 2341.1659
Validation Accuracy: 96.84%
Epoch [11/15], Loss: 2339.2536
Validation Accuracy: 94.74%
Epoch [12/15], Loss: 2337.6042
Validation Accuracy: 95.79%
Epoch [13/15], Loss: 2336.2158
Validation Accuracy: 89.47%
Epoch [14/15], Loss: 2334.8472
Validation Accuracy: 88.42%
Epoch [15/15], Loss: 2334.1785
Validation Accuracy: 93.68%
Model saved to lenet5_trained_model.pth


In [6]:
def evaluate_lenet(model, test_loader):
    correct_test = 0
    total_test = 0
    
    model.eval()  # Set model to evaluation mode
    
    with torch.no_grad():  # Disable gradient computation for inference
        for images_batch_test, labels_batch_test in test_loader:
            outputs_test = model(images_batch_test)
            _, predicted_test_classifications = torch.max(outputs_test.data, 1)   # Get predicted class
            
            total_test += labels_batch_test.size(0)
            correct_test += (predicted_test_classifications == labels_batch_test).sum().item()
    
    print(f'Accuracy of the network on final test images: {100 * correct_test / total_test:.2f}%')

# Evaluate on final test set after training is complete
evaluate_lenet(lenet_model_16x16, test_loader=test_loader)

Accuracy of the network on final test images: 91.58%


In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, ConcatDataset
from torchvision import datasets, transforms
import numpy as np

# Define LeNet5 architecture modified for 16x16 images with dropout
class LeNet5_16x16(nn.Module):
    def __init__(self, num_classes=10):
        super(LeNet5_16x16, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 6, kernel_size=5),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2),
            nn.Dropout(0.25)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(6, 16, kernel_size=5),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2),
            nn.Dropout(0.25)
        )
        self.fc1 = nn.Linear(16 * 1 * 1, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, num_classes)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = torch.tanh(self.fc1(out))
        out = torch.tanh(self.fc2(out))
        out = self.fc3(out)
        return out

# Data augmentation and normalization
transform = transforms.Compose([
    transforms.Resize((16, 16)),
    transforms.RandomAffine(degrees=10, translate=(0.1, 0.1), scale=(0.9, 1.1)),
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

# Load MNIST dataset
mnist_train = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
mnist_test = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# Convert MNIST datasets to TensorDataset
mnist_train_data = torch.stack([img for img, _ in mnist_train])
mnist_train_labels = torch.tensor([label for _, label in mnist_train])
mnist_train_tensor = TensorDataset(mnist_train_data, mnist_train_labels)

mnist_test_data = torch.stack([img for img, _ in mnist_test])
mnist_test_labels = torch.tensor([label for _, label in mnist_test])
mnist_test_tensor = TensorDataset(mnist_test_data, mnist_test_labels)

# Combine MNIST and experimental data
exp_train_dataset = TensorDataset(exp_train_images, exp_train_labels)
val_dataset = TensorDataset(val_images, val_labels)

combined_train = ConcatDataset([mnist_train_tensor, exp_train_dataset])
combined_val = ConcatDataset([mnist_test_tensor, val_dataset])

# Create DataLoaders
train_loader = DataLoader(combined_train, batch_size=64, shuffle=True)
val_loader = DataLoader(combined_val, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Calculate class weights
def calculate_class_weights(combined_dataset):
    all_labels = []
    for dataset in combined_dataset.datasets:
        if isinstance(dataset, TensorDataset):
            all_labels.extend(dataset.tensors[1].tolist())
        else:
            all_labels.extend([label for _, label in dataset])
    
    labels = torch.tensor(all_labels)
    class_counts = torch.bincount(labels)
    class_weights = 1. / class_counts.float()
    return class_weights / class_weights.sum()

class_weights = calculate_class_weights(combined_train)


# Fine-tuning function for LeNet5
def fine_tune_lenet(model, train_loader, val_loader, num_epochs=50, save_path="lenet5_trained_model1.pth"):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    
    criterion = nn.CrossEntropyLoss(weight=class_weights.to(device))
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5, factor=0.5)
    
    best_val_loss = float('inf')
    patience = 10
    counter = 0
    
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
        
        # Validation
        model.eval()
        val_loss = 0.0
        correct_val = 0
        total_val = 0
        
        with torch.no_grad():
            for val_images, val_labels in val_loader:
                val_images, val_labels = val_images.to(device), val_labels.to(device)
                outputs_val = model(val_images)
                loss_val = criterion(outputs_val, val_labels)
                val_loss += loss_val.item()
                
                _, predicted_val = torch.max(outputs_val.data, 1)
                total_val += val_labels.size(0)
                correct_val += (predicted_val == val_labels).sum().item()
        
        val_loss /= len(val_loader)
        val_accuracy = 100 * correct_val / total_val
        
        print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {running_loss/len(train_loader):.4f}, '
              f'Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%')
        
        scheduler.step(val_loss)
        
        # Early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            counter = 0
            torch.save(model.state_dict(), save_path)
            print(f"Model saved to {save_path}")
        else:
            counter += 1
            if counter >= patience:
                print("Early stopping")
                break
    
    return model

# Train the model
lenet_model_16x16 = LeNet5_16x16()
trained_model = fine_tune_lenet(lenet_model_16x16, train_loader, val_loader)

# Evaluate on test set
def evaluate_lenet(model, test_loader):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    model.eval()
    
    correct_test = 0
    total_test = 0
    
    with torch.no_grad():
        for images_batch_test, labels_batch_test in test_loader:
            images_batch_test, labels_batch_test = images_batch_test.to(device), labels_batch_test.to(device)
            outputs_test = model(images_batch_test)
            _, predicted_test_classifications = torch.max(outputs_test.data, 1)
            
            total_test += labels_batch_test.size(0)
            correct_test += (predicted_test_classifications == labels_batch_test).sum().item()
    
    print(f'Accuracy of the network on final test images: {100 * correct_test / total_test:.2f}%')

# Evaluate on final test set
evaluate_lenet(trained_model, test_loader)

Epoch [1/50], Train Loss: 1.6164, Val Loss: 0.6012, Val Accuracy: 81.60%
Model saved to lenet5_trained_model1.pth
Epoch [2/50], Train Loss: 1.3542, Val Loss: 0.4199, Val Accuracy: 87.36%
Model saved to lenet5_trained_model1.pth
Epoch [3/50], Train Loss: 1.2854, Val Loss: 0.3513, Val Accuracy: 89.23%
Model saved to lenet5_trained_model1.pth
Epoch [4/50], Train Loss: 1.2551, Val Loss: 0.3177, Val Accuracy: 90.09%
Model saved to lenet5_trained_model1.pth
Epoch [5/50], Train Loss: 1.2307, Val Loss: 0.2901, Val Accuracy: 91.12%
Model saved to lenet5_trained_model1.pth
Epoch [6/50], Train Loss: 1.2169, Val Loss: 0.2750, Val Accuracy: 91.65%
Model saved to lenet5_trained_model1.pth
Epoch [7/50], Train Loss: 1.2057, Val Loss: 0.2802, Val Accuracy: 91.27%
Epoch [8/50], Train Loss: 1.1966, Val Loss: 0.2614, Val Accuracy: 91.98%
Model saved to lenet5_trained_model1.pth
Epoch [9/50], Train Loss: 1.1919, Val Loss: 0.2509, Val Accuracy: 92.34%
Model saved to lenet5_trained_model1.pth
Epoch [10/50], 

In [9]:
from sklearn.model_selection import KFold

# Data augmentation and normalization
transform = transforms.Compose([
    transforms.Resize((16, 16)),
    transforms.RandomAffine(degrees=10, translate=(0.1, 0.1), scale=(0.9, 1.1)),
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

# Combine all data
all_data = ConcatDataset([mnist_train_tensor, mnist_test_tensor, exp_train_dataset, val_dataset, test_dataset])

# K-Fold Cross-validation
k_folds = 5
kfold = KFold(n_splits=k_folds, shuffle=True, random_state=42)

# Training function
def train_model(model, train_loader, val_loader, num_epochs=30):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)  # Added L2 regularization
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3, factor=0.1)
    
    best_val_loss = float('inf')
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
        
        # Validation
        model.eval()
        val_loss = 0.0
        correct_val = 0
        total_val = 0
        
        with torch.no_grad():
            for val_images, val_labels in val_loader:
                val_images, val_labels = val_images.to(device), val_labels.to(device)
                outputs_val = model(val_images)
                loss_val = criterion(outputs_val, val_labels)
                val_loss += loss_val.item()
                
                _, predicted_val = torch.max(outputs_val.data, 1)
                total_val += val_labels.size(0)
                correct_val += (predicted_val == val_labels).sum().item()
        
        val_loss /= len(val_loader)
        val_accuracy = 100 * correct_val / total_val
        
        print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {running_loss/len(train_loader):.4f}, '
              f'Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%')
        
        scheduler.step(val_loss)
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), "best_model.pth")
    
    return model

# K-Fold Cross-validation training
for fold, (train_ids, val_ids) in enumerate(kfold.split(all_data)):
    print(f'FOLD {fold}')
    print('--------------------------------')
    
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    val_subsampler = torch.utils.data.SubsetRandomSampler(val_ids)
    
    train_loader = DataLoader(all_data, batch_size=32, sampler=train_subsampler)
    val_loader = DataLoader(all_data, batch_size=32, sampler=val_subsampler)
    
    model = LeNet5_16x16()
    train_model(model, train_loader, val_loader)

# Load the best model and evaluate on the test set
best_model = LeNet5_16x16()
best_model.load_state_dict(torch.load("best_model.pth"))
evaluate_lenet(best_model, test_loader)

FOLD 0
--------------------------------
Epoch [1/30], Train Loss: 1.5198, Val Loss: 1.1170, Val Accuracy: 60.23%
Epoch [2/30], Train Loss: 1.2720, Val Loss: 1.0247, Val Accuracy: 62.53%
Epoch [3/30], Train Loss: 1.2170, Val Loss: 0.9742, Val Accuracy: 64.31%
Epoch [4/30], Train Loss: 1.1824, Val Loss: 0.9545, Val Accuracy: 64.88%
Epoch [5/30], Train Loss: 1.1670, Val Loss: 0.9389, Val Accuracy: 65.12%
Epoch [6/30], Train Loss: 1.1515, Val Loss: 0.9304, Val Accuracy: 65.46%
Epoch [7/30], Train Loss: 1.1402, Val Loss: 0.9324, Val Accuracy: 65.49%
Epoch [8/30], Train Loss: 1.1349, Val Loss: 0.9224, Val Accuracy: 65.50%
Epoch [9/30], Train Loss: 1.1291, Val Loss: 0.9100, Val Accuracy: 65.85%
Epoch [10/30], Train Loss: 1.1224, Val Loss: 0.9133, Val Accuracy: 65.55%
Epoch [11/30], Train Loss: 1.1169, Val Loss: 0.9139, Val Accuracy: 65.90%
Epoch [12/30], Train Loss: 1.1123, Val Loss: 0.9043, Val Accuracy: 66.27%
Epoch [13/30], Train Loss: 1.1086, Val Loss: 0.8936, Val Accuracy: 66.60%
Epoch [

C:\Users\Sai\AppData\Local\Temp\ipykernel_50692\4031194117.py:90: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  best_model.load_state_dict(torch.load("best_model.pth"))


In [10]:
class LeNet5_16x16(nn.Module):
    def __init__(self, num_classes=10):
        super(LeNet5_16x16, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 6, kernel_size=5, padding=2),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2, stride=2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(6, 16, kernel_size=5),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2, stride=2)
        )
        self.fc1 = nn.Linear(16 * 4 * 4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, num_classes)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = torch.tanh(self.fc1(out))
        out = torch.tanh(self.fc2(out))
        out = self.fc3(out)
        return out

In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, ConcatDataset, Dataset
from torchvision import datasets, transforms
from sklearn.model_selection import KFold
import numpy as np
import os
from PIL import Image, ImageOps

# Data augmentation and normalization
transform = transforms.Compose([
    transforms.Resize((16, 16)),
    transforms.RandomAffine(degrees=15, translate=(0.1, 0.1), scale=(0.9, 1.1)),
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

# Transform for PIL Images
pil_transform = transforms.Compose([
    transforms.Resize((16, 16)),
    transforms.RandomAffine(degrees=15, translate=(0.1, 0.1), scale=(0.9, 1.1)),
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

# Transform for tensors
tensor_transform = transforms.Compose([
    transforms.Resize((16, 16)),
    transforms.RandomAffine(degrees=15, translate=(0.1, 0.1), scale=(0.9, 1.1)),
    transforms.Normalize((0.1307,), (0.3081,))
])

class CompositeDataset(Dataset):
    def __init__(self, image_paths, labels):
        self.image_paths = image_paths
        self.labels = labels
        
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert('L')
        image = transform(image)
        return image, self.labels[idx]

def load_composite_data(base_folder):
    standard_images, standard_labels = [], []
    rotated_images, rotated_labels = [], []
    cursive_images, cursive_labels = [], []

    for folder in os.listdir(base_folder):
        if folder.startswith('composite_images_'):
            folder_path = os.path.join(base_folder, folder)
            for filename in os.listdir(folder_path):
                if filename.endswith('.png'):
                    file_path = os.path.join(folder_path, filename)
                    digit = int(filename.split('_')[2].split('.')[0])
                    
                    if 'cursive' in filename:
                        cursive_images.append(file_path)
                        cursive_labels.append(digit)
                    elif 'angle' in filename:
                        rotated_images.append(file_path)
                        rotated_labels.append(digit)
                    else:
                        standard_images.append(file_path)
                        standard_labels.append(digit)

    return (
        CompositeDataset(standard_images, standard_labels),
        CompositeDataset(rotated_images, rotated_labels),
        CompositeDataset(cursive_images, cursive_labels)
    )

# Modify MNIST loading
def load_and_invert_mnist():
    mnist_train = datasets.MNIST(root='./data', train=True, download=True, transform=pil_transform)
    mnist_test = datasets.MNIST(root='./data', train=False, download=True, transform=pil_transform)
    
    # Invert half of the MNIST data
    for dataset in [mnist_train, mnist_test]:
        for idx in range(len(dataset.data) // 2):
            dataset.data[idx] = 255 - dataset.data[idx]
    
    return mnist_train, mnist_test

# Load MNIST data
mnist_train, mnist_test = load_and_invert_mnist()

# Ensure experimental data is also [1, 16, 16]
exp_train_images = tensor_transform(exp_train_images)
val_images = tensor_transform(val_images)

# Modify the CompositeDataset __getitem__ method
class CompositeDataset(Dataset):
    def __init__(self, image_paths, labels):
        self.image_paths = image_paths
        self.labels = labels
        
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert('L')
        image = pil_transform(image)
        return image, self.labels[idx]

# Load composite image data
composite_standard, composite_rotated, composite_cursive = load_composite_data('composite_image_folders')

# Combine all data
all_train_data = ConcatDataset([
    mnist_train,
    mnist_test,
    TensorDataset(exp_train_images, exp_train_labels),
    TensorDataset(val_images, val_labels),
    test_dataset
])

mnist_train, mnist_test = load_and_invert_mnist()

# Load experimental data (keep your existing code for loading exp_train_images, exp_train_labels, etc.)

# Load composite image data
composite_standard, composite_rotated, composite_cursive = load_composite_data('composite_image_folders')

# Use composite images as final validation set
final_val_data = ConcatDataset([composite_standard, composite_rotated, composite_cursive])

# K-Fold Cross-validation
k_folds = 5
kfold = KFold(n_splits=k_folds, shuffle=True, random_state=42)

# Training function with early stopping
def train_model(model, train_loader, val_loader, num_epochs=100, patience=10):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5, factor=0.1)
    
    best_val_loss = float('inf')
    counter = 0
    
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
        
        # Validation
        model.eval()
        val_loss = 0.0
        correct_val = 0
        total_val = 0
        
        with torch.no_grad():
            for val_images, val_labels in val_loader:
                val_images, val_labels = val_images.to(device), val_labels.to(device)
                outputs_val = model(val_images)
                loss_val = criterion(outputs_val, val_labels)
                val_loss += loss_val.item()
                
                _, predicted_val = torch.max(outputs_val.data, 1)
                total_val += val_labels.size(0)
                correct_val += (predicted_val == val_labels).sum().item()
        
        val_loss /= len(val_loader)
        val_accuracy = 100 * correct_val / total_val
        
        print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {running_loss/len(train_loader):.4f}, '
              f'Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%')
        
        scheduler.step(val_loss)
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), "best_model_early.pth")
            counter = 0
        else:
            counter += 1
            if counter >= patience:
                print(f"Early stopping at epoch {epoch+1}")
                break
    
    return model

# K-Fold Cross-validation training
for fold, (train_ids, val_ids) in enumerate(kfold.split(all_train_data)):
    print(f'FOLD {fold+1}')
    print('--------------------------------')
    
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    val_subsampler = torch.utils.data.SubsetRandomSampler(val_ids)
    
    train_loader = DataLoader(all_train_data, batch_size=64, sampler=train_subsampler)
    val_loader = DataLoader(all_train_data, batch_size=64, sampler=val_subsampler)
    
    model = LeNet5_16x16()
    train_model(model, train_loader, val_loader)

# Load the best model and evaluate on the final validation set
best_model = LeNet5_16x16()
best_model.load_state_dict(torch.load("best_model_done.pth"))
final_val_loader = DataLoader(final_val_data, batch_size=64, shuffle=False)
evaluate_lenet(best_model, final_val_loader)

FOLD 1
--------------------------------


TypeError: expected Tensor as element 2 in argument 0, but got int